# Analysis of so6 air traffic file

#### <br> Load the so6 file as a pandas dataframe

In [1]:
import pandas as pd
from datetime import datetime

YOUR_SO6_FILE_PATH = 'your_so6_file.so6' 

columns = ['segment_identifier', 'flight_origin', 'flight_destination', 'aircraft_type', 
           'time_begin', 'time_end', 'fl_begin', 'fl_end', 'status', 'callsign', 
           'date_begin', 'date_end', 'lat_begin', 'lon_begin', 'lat_end', 'lon_end',
           'flight_identifier', 'sequence', 'length', 'parity']

parser = lambda x,y: pd.datetime.strptime(x + y, '%y%m%d%H%M%S')
df = pd.read_csv(YOUR_SO6_FILE_PATH, sep=' ', header=None, names=columns, 
                 parse_dates={'datetime_begin': ['date_begin','time_begin'], 
                              'datetime_end': ['date_end','time_end']}, date_parser=parser)

coord_cols = ['lat_begin','lon_begin','lat_end','lon_end']
df[coord_cols] = df[coord_cols].apply(lambda x: x / 60)

traffic_df = df[['callsign','datetime_begin','datetime_end','flight_origin','flight_destination',
                'fl_begin', 'fl_end', 'lat_begin', 'lat_end', 'lon_begin', 'lon_end', 
                 'aircraft_type','length','flight_identifier']]
traffic_df = traffic_df.query('length > 0') # filter null segments
traffic_df.head()

,callsign,datetime_begin,datetime_end,flight_origin,flight_destination,fl_begin,fl_end,lat_begin,lat_end,lon_begin,lon_end,aircraft_type,length,flight_identifier
0,HKS147,2018-03-01 14:25:00,2018-03-01 14:25:44,ENXU,ENXN,2,1,59.445833,59.462500,2.344444,2.365278,S92,1.184730,215700734
3,RYR8005,2018-03-01 06:41:00,2018-03-01 06:41:45,LEAL,EGPD,1,13,38.282222,38.275278,-0.558056,-0.525556,B738,1.586456,215689750
4,RYR8005,2018-03-01 06:41:45,2018-03-01 06:41:55,LEAL,EGPD,13,20,38.275278,38.273056,-0.525556,-0.514722,B738,0.527418,215689750
5,RYR8005,2018-03-01 06:41:55,2018-03-01 06:42:03,LEAL,EGPD,20,20,38.273056,38.270556,-0.514722,-0.503889,B738,0.531892,215689750
6,RYR8005,2018-03-01 06:42:03,2018-03-01 06:42:18,LEAL,EGPD,20,25,38.270556,38.266111,-0.503889,-0.481944,B738,1.067580,215689750


In [2]:
print("Size of traffic table: ", traffic_df.shape)

Size of traffic table:  (1814103, 14)


#### <br> Number of flights in the so6 file

In [5]:
groups = traffic_df.groupby('flight_identifier') # callsign is not unique
print("{} flights in so6 file".format(len(groups)))

25692 flights in so6 file


#### <br> Number of segments per flight

In [6]:
gsize = groups.size()
print("Number of 4D segments per flight: min={} max={} mean={}".format(gsize.min(), gsize.max(), gsize.mean()))

Number of 4D segments per flight: min=1 max=138 mean=47.39265140899891


#### <br> Length of flights (in nautical miles)

In [8]:
glen = groups['length'].sum()
print("Length of flights: min={} max={} mean={}".format(glen.min(), glen.max(), glen.mean()))

Length of flights: min=0.292157 max=7979.490984999998 mean=1031.4481642298122


#### <br> Display the longest flight

In [9]:
traffic_df[traffic_df['flight_identifier'] == glen.idxmax()].head() # remove .head() to get all 4D segments

,callsign,datetime_begin,datetime_end,flight_origin,flight_destination,fl_begin,fl_end,lat_begin,lat_end,lon_begin,lon_end,aircraft_type,length,flight_identifier
787917,CSN300,2018-03-01 16:15:00,2018-03-01 16:15:05,KJFK,ZGGG,0,25,40.640278,40.700556,-73.778333,-73.752778,B77W,3.799054,215702552
787918,CSN300,2018-03-01 16:15:05,2018-03-01 16:15:28,KJFK,ZGGG,25,35,40.700556,40.717778,-73.752778,-73.745556,B77W,1.084287,215702552
787919,CSN300,2018-03-01 16:15:28,2018-03-01 16:15:58,KJFK,ZGGG,35,50,40.717778,40.751944,-73.745556,-73.730833,B77W,2.156504,215702552
787920,CSN300,2018-03-01 16:15:58,2018-03-01 16:16:38,KJFK,ZGGG,50,70,40.751944,40.803611,-73.730833,-73.709167,B77W,3.252551,215702552
787921,CSN300,2018-03-01 16:16:38,2018-03-01 16:18:00,KJFK,ZGGG,70,110,40.803611,40.924167,-73.709167,-73.658056,B77W,7.596043,215702552


#### <br> Display the top 10 aicraft types used in flights

In [9]:
groups.first()['aircraft_type'].value_counts().nlargest(10)

B738    4595
A320    4075
A319    1925
A321    1569
E190     662
DH8D     649
B77W     553
CRJ9     474
B737     462
A332     427
Name: aircraft_type, dtype: int64

#### <br> Top 5 origin airports

In [20]:
groups.first()['flight_origin'].value_counts().nlargest(5)

EDDF    655
LTBA    633
LFPG    631
EHAM    604
EGLL    583
Name: flight_origin, dtype: int64

#### <br> Top 5 destination airports

In [21]:
groups.first()['flight_destination'].value_counts().nlargest(5)

EDDF    644
LFPG    630
LTBA    612
EHAM    592
LEMD    554
Name: flight_destination, dtype: int64

#### <br> Number of flights from one airport to another

In [30]:
lfbo_lfpo = groups.first().query('flight_origin=="LFBO" and flight_destination=="LFPO"').index
print("Nb of flights from LFBO (Toulouse-Blagnac) to LFPO (Paris-Orly): {}".format(len(lfbo_lfpo)))

Nb of flights from LFBO (Toulouse-Blagnac) to LFPO (Paris-Orly): 23


#### <br> Most frequent city pair

In [16]:
from_to = groups.first().groupby(['flight_origin', 'flight_destination']).size()
print("{} flights for the top city pair: {}".format(from_to.max(), from_to.idxmax()))

37 flights for the top city pair: ('ENGM', 'ENVA')


#### <br> Display all destinations from one given airport

In [56]:
groups.first().query('flight_origin=="LFBO"')['flight_destination'].unique()

array(['EBLG', 'EGSS', 'LFPG', 'EDDM', 'LFPO', 'LFRS', 'EHAM', 'EDDS',
       'EGGD', 'EBBR', 'EDHI', 'EDDF', 'LPPT', 'EGKK', 'EGNR', 'EGLL',
       'LIPZ', 'LEMD', 'LFLL', 'LFQQ', 'LEVT', 'LFST', 'LMML', 'GMMN',
       'LFMK', 'DTTA', 'DAOO', 'LIRF', 'DAAG', 'EBCI', 'GCRR', 'VIDP',
       'LSGG', 'LFML', 'LFCR', 'GMAD', 'EDNY', 'LFRK', 'LFRN', 'LFMT',
       'LEZL', 'EGCC', 'LFMN', 'LFLX', 'UACC', 'EDDB', 'LFSB', 'LEBL',
       'EGGW'], dtype=object)